Importing Libraries

In [1]:
pip install slack_sdk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd
import time
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from datetime import datetime, timedelta
from IPython.display import clear_output, display

Initialize the Firebase Admin SDK with the service account

In [3]:
cred = credentials.Certificate('data/siotch-firebase-adminsdk-pfw4q-dbd701b65c.json')  # Service account key path
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://siotch-default-rtdb.europe-west1.firebasedatabase.app/'  # Your Firebase Realtime Database URL
})

Reference to the Firebase Realtime Database

In [4]:
ref = db.reference('/')

# Fetch the data from Firebase
data = ref.get()

Adjusting Timestamps (initialised when esp-32s were deployed)

In [5]:
# Define the actual export time
export_time = datetime.strptime("2024-12-11 15:40:00", "%Y-%m-%d %H:%M:%S")

# Define the erroneous end time
end_time = datetime.strptime("2000-01-10 22:00:49", "%Y-%m-%d %H:%M:%S")

# Calculate the time delta
time_delta = export_time - end_time

# Correct the timestamps
for key, entry in data.get("air_quality", {}).items():  # Accessing the nested 'air_quality' key
    # Check if 'timestamp' exists in the entry
    if 'timestamp' in entry:
        erroneous_timestamp = datetime.strptime(entry["timestamp"], "%Y-%m-%d %H:%M:%S")
        corrected_timestamp = erroneous_timestamp + time_delta
        entry["timestamp"] = corrected_timestamp.strftime("%Y-%m-%d %H:%M:%S")

Slack API running code, sending message if PM2.5 is too high!

In [ ]:
# Slack API Authentication
slack_token = ''  # Private Token
channel_id = 'C084QRYAEBF'  # Private channel ID (ensure this is the correct channel ID)

client = WebClient(token=slack_token)

# Fetch the data from Firebase Realtime Database
def fetch_data_from_firebase():
    try:
        print("Fetching data from Firebase...")
        ref = db.reference('/')  # Reference to the root of the database
        data = ref.get()  # Fetch the data

        if data is None:
            print("No data found in Firebase.")
            return pd.DataFrame()  # Return an empty DataFrame if no data exists

        print("Data fetched successfully from Firebase.")
        
        # Convert the fetched data to a pandas DataFrame
        records = []
        for entry in data.get("air_quality", {}).values():  # Accessing the nested 'air_quality' key
            # Check if the required keys exist
            if "timestamp" in entry and "arch" in entry:
                records.append({
                    "timestamp": entry["timestamp"],
                    "arch": entry["arch"],
                    "pm25": entry.get("pm25", None),  # Use .get() to avoid KeyError if 'pm25' doesn't exist
                    "temperature": entry.get("temperature", None),
                    "humidity": entry.get("humidity", None)
                })
        
        if not records:
            print("No valid records found in the 'air_quality' data.")
        
        df = pd.DataFrame(records)
        return df
    except Exception as e:
        print(f"Error fetching data from Firebase: {e}")
        return pd.DataFrame()  # Return empty DataFrame on error

# Function to output PM2.5 readings for all arches
def print_all_pm25(df):
    if df.empty:
        print("No data to display.")
        return

    # Convert timestamp to datetime (no unit specified since it's already a date string)
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')  # Use 'coerce' to handle invalid timestamps gracefully

    # Print PM2.5 readings in the format you'd send to Slack
    print("⚠️ PM2.5 Data from the last run:")
    for _, row in df.iterrows():
        print(f"Arch: {row['arch']}, PM2.5: {row['pm25']} µg/m³")

    # Simulate sending a Slack message if PM2.5 exceeds threshold
    high_pm25_arches = df[df['pm25'] > 5]['arch'].tolist()
    if high_pm25_arches:
        message = f"⚠️ Beware, high air quality alerts in the following arches: {','.join(high_pm25_arches)}"
        print(message)  # Print the message you would send to Slack
    else:
        print("All PM2.5 levels are within safe limits.")
        
# Function to send a message to Slack
def send_slack_message(message):
    try:
        # Send a message to the Slack channel using channel ID (not channel name)
        response = client.chat_postMessage(channel=channel_id, text=message)
        print(f"Message sent to Slack: {response['message']['text']}")
    except SlackApiError as e:
        print(f"Error sending message to Slack: {e.response['error']}")

# Main loop that runs every 5 minutes and updates the notebook
def run_periodically():
    print("Starting periodic monitoring of PM2.5 levels...")

    # For testing, we'll run it for a set number of cycles (e.g., 5 cycles)
    for _ in range(5):
        
        # Fetch data from Firebase
        df = fetch_data_from_firebase()  # Actual Firebase data fetch
        
        if df.empty:
            print("Skipping this cycle due to empty data.\n")
        else:
            # Print PM2.5 data for all arches
            print_all_pm25(df)

            # Optional: Display the full DataFrame for debugging/insight
            print("\nFull DataFrame:")
            display(df)

        # Wait for 5 minutes (300 seconds) before checking again
        print("\nWaiting for the next cycle...\n")
        time.sleep(300)

run_periodically()

Message sent to Slack: :warning: Beware, high air quality alerts in the following arches: A
Starting periodic monitoring of PM2.5 levels...
Fetching data from Firebase...
Data fetched successfully from Firebase.
⚠️ PM2.5 Data from the last run:
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: 0.0 µg/m³
Arch: Arch C, PM2.5: 5.0 µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: 0.0 µg/m³
Arch: Arch C, PM2.5: 3.0 µg/m³
Arch: Arch B, PM2.5: 0.0 µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 3.0 µg/m³
Arch: Arch B, PM2.5: 0.0 µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 3.0 µg/m³
Arch: Arch B, PM2.5: 1.0 µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 2.0 µg/m³
Arch: Arch B, PM2.5: 1.0 µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 2.0 µg/m³
Arch: Arch B, PM2.5: 1.0 µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 2.0 µg/m³
Arch: Arch B, PM2.5: 1.0 µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 2.0 µg/m³
Arch: Arch 

Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Ar

Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 3.0 µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Ar

Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 2.0 µg/m³
Arch: Arch B, PM2.5: 0.0 µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 2.0 µg/m³
Arch: Arch B, PM2.5: 0.0 µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 2.0 µg/m³
Arch: Arch B, PM2.5: 0.0 µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 2.0 µg/m³
Arch: Arch B, PM2.5: 0.0 µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 2.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 2.0 µg/m³
Arch: Arch B, PM2.5: 0.0 µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 2.0 µg/m³
Arch: Arch B, PM2.5: 0.0 µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 2.0 µg/m³
Arch: Arch B, PM2.5: 0.0 µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 2.0 µg/m³
Arch: Arch B, PM2.5: 0.0 µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 2.0 µg/m³
Arch: Arch B, PM2.5: 0.0 µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 2.0 µg/m³
Arch: Ar

,timestamp,arch,pm25,temperature,humidity
0,2000-01-01 00:36:30,Arch A,NaN,18.59,60.29
1,2000-01-01 00:21:25,Arch B,0.0,20.51,55.05
2,2000-01-01 00:04:38,Arch C,5.0,20.56,56.18
3,2000-01-01 00:37:01,Arch A,NaN,18.55,60.13
4,2000-01-01 00:21:56,Arch B,0.0,20.52,55.14
...,...,...,...,...,...
15028,2000-01-11 17:27:37,Arch C,NaN,18.79,42.59
15029,2000-01-11 17:44:43,Arch B,0.0,19.27,41.80
15030,2000-01-11 17:59:55,Arch A,NaN,16.50,44.46
15031,2000-01-11 17:28:09,Arch C,NaN,18.77,42.57


...

Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 0.0 µg/m³
Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 0.0 µg/m³
Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 0.0 µg/m³
Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: 0.0 µg/m³
Arch: Arch C, PM2.5: 0.0 µg/m³
Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: 0.0 µg/m³
Arch: Arch C, PM2.5: 0.0 µg/m³
Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: 0.0 µg/m³
Arch: Arch C, PM2.5: 0.0 µg/m³
Arch: Arch A, PM2.5: 1.0 µg/m³
Arch: Arch B, PM2.5: 0.0 µg/m³
Arch: Arch C, PM2.5: 0.0 µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: 0.0 µg/m³
Arch: Ar

Arch: Arch C, PM2.5: 0.0 µg/m³
Arch: Arch A, PM2.5: 6.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 0.0 µg/m³
Arch: Arch A, PM2.5: 6.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 0.0 µg/m³
Arch: Arch A, PM2.5: 6.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 6.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: 6.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Ar

Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: nan µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 0.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 0.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 0.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 0.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 0.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 0.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 0.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 0.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Arch A, PM2.5: nan µg/m³
Arch: Arch C, PM2.5: 0.0 µg/m³
Arch: Arch B, PM2.5: nan µg/m³
Arch: Ar

,timestamp,arch,pm25,temperature,humidity
0,2000-01-01 00:36:30,Arch A,NaN,18.59,60.29
1,2000-01-01 00:21:25,Arch B,0.0,20.51,55.05
2,2000-01-01 00:04:38,Arch C,5.0,20.56,56.18
3,2000-01-01 00:37:01,Arch A,NaN,18.55,60.13
4,2000-01-01 00:21:56,Arch B,0.0,20.52,55.14
...,...,...,...,...,...
15115,2000-01-11 18:14:27,Arch A,NaN,16.65,44.75
15116,2000-01-11 17:59:29,Arch B,NaN,19.30,42.01
15117,2000-01-11 17:42:59,Arch C,0.0,18.76,42.87
15118,2000-01-11 18:14:58,Arch A,NaN,16.65,44.78



Waiting for the next cycle...

